In [ ]:
import csv
import os
import pandas as pd
import glob
import re

In [ ]:
path = "C:/kodowanko/assecco/zzzklienci/"
all_files = [path + "HACK_OPERACJE_DATA_TABLE.csv", path + "RACHUNKI.csv", path + "KLIENCI.csv",  path + "KARTY.csv"]
#Frames: 0-data_table, 1-rachunki, 2-klienci, 3-karty
frames = []

for filename in all_files:
    #print("Df zrobiony z - > " + filename)
    df = pd.read_csv(filename, encoding_errors = 'ignore')
    frames.append(df)

In [ ]:
#potrzbne: [0, 1, 4, 10] ["DATA_OPERACJI", "ID_UMOWY", "KWOTA_OBCIAZENIA", "KATEGORIA_WLASCIWA"]
frames[0] = frames[0].drop(columns=["KANAL", "KOD_OPERACJI", "KWOTA_UZNANIA", "TYTUL", "BENEFICJENT", 
                                    "ZLECENIODAWCA", "KURS_OBCIAZENIA", "KURS_UZNANIA", "KATEGORIA_WG_TYTULU", "NADKATEGORIA"])
frames[1] = frames[1].drop(columns=["KOD_PRODUKTU", "WALUTA", "DATA_ZAKONCZENIA"])

frames[2] = frames[2].drop(columns=["MIEJSCE_URODZENIA", "ADRES_KRAJ", "ADRES_GMINA", "DATA_AKT_DANYCH",
                                    "PKD", "DATA_POWST_FIRMY", "FORMA_PRAWNA","LICZBA_ZATRUDNIONYCH"])


In [ ]:
#print(frames[0])
#print(frames[1])
#print(frames[2])

In [ ]:
df = frames[0].merge(frames[1], how="inner", on="ID_UMOWY")

In [ ]:
df = df.merge(frames[2], how="inner", on="ID_KLIENTA")

In [ ]:
df[df.TYP_KLIENTA == 15]
df[df.RODZAJ_KLIENTA == "O"]

In [ ]:
def data2int(s: str) -> int:
    if type(s) == str:
        return int(2021 - 1900 - int(s.split('/')[0]))
    else:
        return 0

In [ ]:
def m(s : str) -> int:
    if type(s) == str:
        return int(s.split('/')[1])
    else:
        return 0

In [ ]:
def plec(s):
    if s == 'M':
        return 2
    elif s == 'K':
        return 1
    else:
        return 0

In [ ]:
def stan(s):
    if s == 'kawaler':
        return 1
    elif s == 'onaty / matka':
        return 2
    elif s == 'Kawaler / pan':
        return 3
    elif s == 'Wdowiec / wdowa':
        return 4
    else:
        return 0

In [ ]:
def wyksz(s):
    if s == 'rednie':
        return 2
    elif s == 'Niepene i podstawowe':
        return 1
    elif s == 'Zasadnicze zawodowe':
        return 3
    elif s == 'wysze':
        return 4
    else:
        return 0

In [ ]:
df['MIESIAC_OPERACJI'] = df['DATA_OPERACJI'].apply(m)
df['WIEK_KLIENTA'] = df['DATA_URODZENIA'].apply(data2int)
df['WIEK_KONTA'] = df['DATA_ZALOZENIA'].apply(data2int)
df['PLEC_2'] = df['PLEC'].apply(plec)
df['STAN_CYWILNY_2'] = df['STAN_CYWILNY'].apply(stan)
df['WYKSZTALCENIE_2'] = df['WYKSZTALCENIE'].apply(wyksz)

In [ ]:
df

In [ ]:
from sklearn.preprocessing import LabelEncoder
labelencoder = LabelEncoder()

label_encoded = df

label_encoded['gdagafsaf'] = labelencoder.fit_transform(label_encoded['KATEGORIA_WLASCIWA'])
df['KATEGORIA_'] = label_encoded['gdagafsaf']
label_encoded

In [ ]:
from sklearn.preprocessing import OneHotEncoder

da = OneHotEncoder(handle_unknown='ignore').fit(X)
enc_df = pd.DataFrame(da.fit_transform(df[['KATEGORIA_WLASCIWA']]).toarray())

one_hot_data = df.join(enc_df)
one_hot_data

In [ ]:
df1 = df[['PLEC_2', 'STAN_CYWILNY_2', 'WYKSZTALCENIE_2', 'KATEGORIA_']]
df2 = df[['KWOTA_OBCIAZENIA', 'MIESIAC_OPERACJI', 'WIEK_KLIENTA', 'WIEK_KONTA']]
df1.to_csv('C:/kodowanko/assecco/zzzklienci/dane_kategoryczne.csv')
df2.to_csv('C:/kodowanko/assecco/zzzklienci/dane_numeryczne.csv')
